In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.metrics import roc_auc_score
import operator
import time
import datetime

In [2]:
def prepareData():
    df = pd.read_csv('./features.csv', index_col='match_id')
    y = df.radiant_win.values[:]
    df.fillna(0, inplace=True)
    df_GB = df.drop(['duration', 'radiant_win', 'tower_status_radiant', 'tower_status_dire', 'barracks_status_radiant', 'barracks_status_dire'], axis = 1)
    return (df_GB, y)

In [3]:
def dataPick(data):
    N = np.max(np.unique(data[['r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero']].values))
    X_pick = np.zeros((data.shape[0], N))

    for i, match_id in enumerate(data.index):
        for p in xrange(5):
            X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
    for i, match_id in enumerate(data.index):
        for p in xrange(5):
            X_pick[i, data.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
            X_pick[i, data.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1
    
    res = data.drop(['lobby_type', 'r1_hero', 'r2_hero', 'r3_hero', 'r4_hero', 'r5_hero', 'd1_hero', 'd2_hero', 'd3_hero', 'd4_hero', 'd5_hero'], axis = 1)
    
    return np.hstack((res.values[:,:], X_pick))

In [4]:
(train_data, train_labels) = prepareData()

In [5]:
train_data.head()

,start_time,lobby_type,r1_hero,r1_level,r1_xp,r1_gold,r1_lh,r1_kills,r1_deaths,r1_items,...,radiant_ward_sentry_count,radiant_first_ward_time,dire_bottle_time,dire_courier_time,dire_flying_courier_time,dire_tpscroll_count,dire_boots_count,dire_ward_observer_count,dire_ward_sentry_count,dire_first_ward_time
match_id,,,,,,,,,,,,,,,,,,,,,
0,1430198770,7,11,5,2098,1489,20,0,0,7,...,0,35,103,-84,221,3,4,2,2,-52
1,1430220345,0,42,4,1188,1033,9,0,1,12,...,0,-20,149,-84,195,5,4,3,1,-5
2,1430227081,7,33,4,1319,1270,22,0,0,12,...,1,-39,45,-77,221,3,4,3,1,13
3,1430263531,1,29,4,1779,1056,14,0,0,5,...,0,-30,124,-80,184,0,4,2,0,27
4,1430282290,7,13,4,1431,1090,8,1,0,8,...,0,46,182,-80,225,6,3,3,0,-16


In [6]:
train_data.shape

(97230, 102)

In [9]:
X = dataPick(train_data)

In [10]:
y = train_labels

# PCA

In [15]:
pca = PCA(n_components=150)

In [19]:
X_PCA = pca.fit_transform(X)

# Random Forest

In [21]:
rfc = RandomForestClassifier(n_estimators=100)

In [ ]:
rfc.fit(X_train, y_train)

# SVC

In [11]:
grid = {'C': np.power(10.0, np.arange(-3, 4))}
cv = KFold(train_labels.size, n_folds=5, shuffle=True, random_state=42)

In [8]:
clf = svm.SVC(kernel='rbf',random_state=42)

In [9]:
gs = GridSearchCV(clf, grid, scoring='roc_auc', cv=cv, verbose=8, n_jobs=4)

In [ ]:
gs.fit(X, y)

Fitting 5 folds for each of 7 candidates, totalling 35 fits
